03_TEXTCNN文本分类

In [1]:
# 显卡查看
! nvidia-smi

Sun Jul  5 04:46:56 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8     8W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# 依赖安装
! pip install fastNLP

加载数据集

In [3]:
import sys

from fastNLP.core import Const

sys.path.insert(0, '/content/drive/My Drive/my_framework/qyt_clue/')  # 定义搜索路径的优先顺序，序号从0开始，表示最大优先级

import myClue  # noqa
print('myClue module path :{}'.format(myClue.__file__))  # 输出测试模块文件位置
from myClue.core import logger  # noqa
from myClue.core.utils import print_data_bundle  # noqa
from myClue.tools.serialize import save_serialize_obj  # noqa
from myClue.loader.classification import THUCNewsLoader  # noqa
from myClue.pipe.classification import THUCNewsPipe  # noqa

myClue module path :/content/drive/My Drive/my_framework/qyt_clue/myClue/__init__.py


In [4]:
train_file_config = {
        'train': '/content/drive/My Drive/game/UCAS_NLP_TC/data_03_50words_split/traindata.txt',
        'dev': '/content/drive/My Drive/game/UCAS_NLP_TC/data_03_50words_split/devdata.txt',
    }

In [5]:
logger.info('数据加载')
data_loader = THUCNewsLoader()
data_bundle = data_loader.load(train_file_config)
print_data_bundle(data_bundle)
logger.info('数据预处理')
data_pipe = THUCNewsPipe(tokenizer='white_space')
data_bundle = data_pipe.process(data_bundle)
data_bundle.rename_field(field_name=Const.CHAR_INPUT, new_field_name=Const.INPUT, ignore_miss_dataset=True, rename_vocab=True)
print_data_bundle(data_bundle)

2020-07-05 04:47:06 I [<ipython-input-5-d23eb2358db5>:1] 数据加载
2020-07-05 04:47:07 I [utils.py:16] dataset name : train
2020-07-05 04:47:07 I [utils.py:17] dataset len : 40227
2020-07-05 04:47:07 I [utils.py:18] dataset example : 
2020-07-05 04:47:07 I [utils.py:19] 
+-------------------------------------------+-------------+
| raw_chars                                 | target      |
+-------------------------------------------+-------------+
| 近日 ， 胡杏儿 以 品牌 挚友 及 公益 大... | EnterSports |
| 刚刚 杀青 ， 片 中 与 古天乐 、 张智霖... | EnterSports |
| 虎 扑 1月 14日 讯 根据 《 世界 体育报...  | EnterSports |
| 的 球员 ， 而且 拉比奥 也 即将 加盟 。... | EnterSports |
| 奥拉 也 是 一样 的 情况 。 该 记者 认...  | EnterSports |
+-------------------------------------------+-------------+
2020-07-05 04:47:07 I [utils.py:20] dataset 输出各个field的被设置成input和target的情况 : 
2020-07-05 04:47:07 I [utils.py:21] 
+-------------+-----------+--------+
| field_names | raw_chars | target |
+-------------+-----------+--------+
|   is_input  |   False   | False  |
|  

+-------------+-----------+--------+
| field_names | raw_chars | target |
+-------------+-----------+--------+
|   is_input  |   False   | False  |
|  is_target  |   False   | False  |
| ignore_type |           |        |
|  pad_value  |           |        |
+-------------+-----------+--------+
+-------------+-----------+--------+
| field_names | raw_chars | target |
+-------------+-----------+--------+
|   is_input  |   False   | False  |
|  is_target  |   False   | False  |
| ignore_type |           |        |
|  pad_value  |           |        |
+-------------+-----------+--------+


2020-07-05 04:47:16 I [utils.py:16] dataset name : train
2020-07-05 04:47:16 I [utils.py:17] dataset len : 40227
2020-07-05 04:47:16 I [utils.py:18] dataset example : 
2020-07-05 04:47:16 I [utils.py:19] 
+--------------------+--------+--------------------+---------+--------------------+
| raw_chars          | target | chars              | seq_len | words              |
+--------------------+--------+--------------------+---------+--------------------+
| 近日 ， 胡杏儿 ... | 4      | ['近日', '，', ... | 50      | [692, 2, 24218,... |
| 刚刚 杀青 ， 片... | 4      | ['刚刚', '杀青'... | 42      | [1353, 9959, 2,... |
| 虎 扑 1月 14日 ... | 4      | ['虎', '扑', '1... | 50      | [2606, 3869, 26... |
| 的 球员 ， 而且... | 4      | ['的', '球员', ... | 50      | [3, 708, 2, 378... |
| 奥拉 也 是 一样... | 4      | ['奥拉', '也', ... | 24      | [35057, 16, 6, ... |
+--------------------+--------+--------------------+---------+--------------------+
2020-07-05 04:47:16 I [utils.py:20] dataset 输出各个field的被设置成input和target的情况 : 
2020-

+-------------+-----------+--------+-----------+---------+-------+
| field_names | raw_chars | target | raw_words | seq_len | words |
+-------------+-----------+--------+-----------+---------+-------+
|   is_input  |   False   | False  |   False   |   True  |  True |
|  is_target  |   False   |  True  |   False   |  False  | False |
| ignore_type |           | False  |           |  False  | False |
|  pad_value  |           |   0    |           |    0    |   0   |
+-------------+-----------+--------+-----------+---------+-------+
+-------------+-----------+--------+-----------+---------+-------+
| field_names | raw_chars | target | raw_words | seq_len | words |
+-------------+-----------+--------+-----------+---------+-------+
|   is_input  |   False   | False  |   False   |   True  |  True |
|  is_target  |   False   |  True  |   False   |  False  | False |
| ignore_type |           | False  |           |  False  | False |
|  pad_value  |           |   0    |           |    0    |   0

模型训练

In [6]:
import os
import sys
import time

import torch

from fastNLP import Const
from fastNLP.embeddings import StaticEmbedding
# from fastNLP.models import CNNText
from fastNLP import Trainer
from fastNLP import CrossEntropyLoss
from torch.optim import Adam
from fastNLP import Tester

from myClue.core.metrics import ClassifyFPreRecMetric  # noqa
from myClue.core.callback import EarlyStopCallback  # noqa
from myClue.tools.serialize import save_serialize_obj  # noqa
from myClue.tools.file import init_file_path  # noqa
from myClue.models import CNNText  # noqa

In [7]:
model_path = '/content/drive/My Drive/game/UCAS_NLP_TC/models/03_train_textcnn_words_split'
init_file_path(model_path)
logger.add_file_handler(os.path.join(model_path, 'log_{}.txt'.format(time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime()))))  # 日志写入文件
char_vocab_pkl_file = os.path.join(model_path, 'vocab_char.pkl')
target_vocab_pkl_file = os.path.join(model_path, 'target_char.pkl')
logger.warn('获取词典')
char_vocab = data_bundle.get_vocab('words')
logger.info('char_vocab:{}'.format(char_vocab))
target_vocab = data_bundle.get_vocab('target')
logger.info('target_vocab:{}'.format(target_vocab))
save_serialize_obj(char_vocab, char_vocab_pkl_file)
save_serialize_obj(target_vocab, target_vocab_pkl_file)
logger.info('词典序列化:{}'.format(char_vocab_pkl_file))
logger.warn('选择预训练词向量')
model_dir_or_name = '/content/drive/My Drive/data/embed/ctb.50d.vec'
word2vec_embed = StaticEmbedding(char_vocab, model_dir_or_name=model_dir_or_name)
logger.warn('神经网络模型')
model = CNNText(word2vec_embed, num_classes=len(target_vocab))
logger.info(model)
logger.warn('训练超参数设定')
loss = CrossEntropyLoss()
optimizer = Adam([param for param in model.parameters() if param.requires_grad])
# metric = AccuracyMetric()
metric = ClassifyFPreRecMetric(tag_vocab=data_bundle.get_vocab(Const.TARGET), only_gross=False)  # 若only_gross=False, 即还会返回各个label的metric统计值
device = 'cuda' if torch.cuda.is_available() else 'cpu'  # 如果有gpu的话在gpu上运行，训练速度会更快
logger.info('device:{}'.format(device))
batch_size = 32
n_epochs = 200
early_stopping = 10
trainer = Trainer(
    save_path=model_path,
    train_data=data_bundle.get_dataset('train'),
    model=model,
    loss=loss,
    optimizer=optimizer,
    batch_size=batch_size,
    n_epochs=n_epochs,
    dev_data=data_bundle.get_dataset('dev'),
    metrics=metric,
    metric_key='f',
    device=device,
    callbacks=[EarlyStopCallback(early_stopping)])
logger.warn('开始训练')
trainer.train()

2020-07-05 04:47:17 W [<ipython-input-7-8f4c01cf6b98>:6] 获取词典
2020-07-05 04:47:17 I [<ipython-input-7-8f4c01cf6b98>:8] char_vocab:Vocabulary(['近日', '，', '胡杏儿', '以', '品牌']...)
2020-07-05 04:47:17 I [<ipython-input-7-8f4c01cf6b98>:10] target_vocab:Vocabulary(['EnterSports', 'Economics', 'Technology', 'Military', 'Government']...)
2020-07-05 04:47:17 I [<ipython-input-7-8f4c01cf6b98>:13] 词典序列化:/content/drive/My Drive/game/UCAS_NLP_TC/models/03_train_textcnn_words_split/vocab_char.pkl
2020-07-05 04:47:17 W [<ipython-input-7-8f4c01cf6b98>:14] 选择预训练词向量


Found 58398 out of 102455 words in the pre-training embedding.


/pytorch/aten/src/ATen/native/TensorFactories.cpp:361: UserWarning: Deprecation warning: In a future PyTorch release torch.full will no longer return tensors of floating dtype by default. Instead, a bool fill_value will return a tensor of torch.bool dtype, and an integral fill_value will return a tensor of torch.long dtype. Set the optional `dtype` or `out` arguments to suppress this warning.
2020-07-05 04:47:23 W [<ipython-input-7-8f4c01cf6b98>:17] 神经网络模型
2020-07-05 04:47:23 I [<ipython-input-7-8f4c01cf6b98>:19] CNNText(
  (embed): Embedding(
    (embed): StaticEmbedding(
      (dropout_layer): Dropout(p=0, inplace=False)
      (embedding): Embedding(96392, 50, padding_idx=0)
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (conv_pool): ConvMaxpool(
    (convs): ModuleList(
      (0): Conv1d(50, 30, kernel_size=(1,), stride=(1,), bias=False)
      (1): Conv1d(50, 40, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (2): Conv1d(50, 50, kernel_size=(5,), stride=(1,

input fields after batch(if batch size is 2):
	seq_len: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 
	words: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2, 50]) 
target fields after batch(if batch size is 2):
	target: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 

training epochs started 2020-07-05-04-47-26-924614


Evaluate data in 0.78 seconds!


...

2020-07-05 04:49:34 W [callback.py:38] ======epoch : 12 , early stopping : 9/10======
2020-07-05 04:49:34 I [callback.py:40] metric_key : f, metric_value : 0.732465
2020-07-05 04:49:34 I [callback.py:41] eval_result : 
{'ClassifyFPreRecMetric': {'f-Economics': 0.705544, 'pre-Economics': 0.622087, 'rec-Economics': 0.814864, 'f-Technology': 0.71914, 'pre-Technology': 0.624307, 'rec-Technology': 0.847945, 'f-Government': 0.61557, 'pre-Government': 0.52106, 'rec-Government': 0.751961, 'f-Military': 0.765844, 'pre-Military': 0.673565, 'rec-Military': 0.887423, 'f-EnterSports': 0.910984, 'pre-EnterSports': 0.856164, 'rec-EnterSports': 0.973304, 'f': 0.732465, 'pre': 0.643729, 'rec': 0.849577}}


Evaluate data in 0.73 seconds!
Evaluation on dev at Epoch 12/200. Step:15096/251600: 
ClassifyFPreRecMetric: f-Economics=0.705544, pre-Economics=0.622087, rec-Economics=0.814864, f-Technology=0.71914, pre-Technology=0.624307, rec-Technology=0.847945, f-Government=0.61557, pre-Government=0.52106, rec-Government=0.751961, f-Military=0.765844, pre-Military=0.673565, rec-Military=0.887423, f-EnterSports=0.910984, pre-EnterSports=0.856164, rec-EnterSports=0.973304, f=0.732465, pre=0.643729, rec=0.849577



2020-07-05 04:49:44 W [callback.py:38] ======epoch : 13 , early stopping : 10/10======
2020-07-05 04:49:44 I [callback.py:40] metric_key : f, metric_value : 0.731781
2020-07-05 04:49:44 I [callback.py:41] eval_result : 
{'ClassifyFPreRecMetric': {'f-Economics': 0.712321, 'pre-Economics': 0.629686, 'rec-Economics': 0.819921, 'f-Technology': 0.708457, 'pre-Technology': 0.604135, 'rec-Technology': 0.856326, 'f-Government': 0.621429, 'pre-Government': 0.531929, 'rec-Government': 0.747137, 'f-Military': 0.756445, 'pre-Military': 0.656208, 'rec-Military': 0.892825, 'f-EnterSports': 0.915479, 'pre-EnterSports': 0.879648, 'rec-EnterSports': 0.954352, 'f': 0.731781, 'pre': 0.642345, 'rec': 0.85015}}
2020-07-05 04:49:44 I [callback.py:46] reach early stopping patience, stop training.
2020-07-05 04:49:44 I [callback.py:55] Early Stopping triggered in epoch 13!


Evaluate data in 0.75 seconds!
Reloaded the best model.

In Epoch:2/Step:2516, got best dev performance:
ClassifyFPreRecMetric: f-Economics=0.725, pre-Economics=0.631712, rec-Economics=0.850614, f-Technology=0.768881, pre-Technology=0.682804, rec-Technology=0.879792, f-Government=0.679348, pre-Government=0.594429, rec-Government=0.792572, f-Military=0.80312, pre-Military=0.721629, rec-Military=0.90536, f-EnterSports=0.928535, pre-EnterSports=0.896282, rec-EnterSports=0.963197, f=0.770682, pre=0.688514, rec=0.87512


{'best_epoch': 2,
 'best_eval': {'ClassifyFPreRecMetric': {'f': 0.770682,
   'f-Economics': 0.725,
   'f-EnterSports': 0.928535,
   'f-Government': 0.679348,
   'f-Military': 0.80312,
   'f-Technology': 0.768881,
   'pre': 0.688514,
   'pre-Economics': 0.631712,
   'pre-EnterSports': 0.896282,
   'pre-Government': 0.594429,
   'pre-Military': 0.721629,
   'pre-Technology': 0.682804,
   'rec': 0.87512,
   'rec-Economics': 0.850614,
   'rec-EnterSports': 0.963197,
   'rec-Government': 0.792572,
   'rec-Military': 0.90536,
   'rec-Technology': 0.879792}},
 'best_step': 2516,
 'seconds': 137.82}

模型预测

In [8]:
import torch
import os
import codecs

from fastNLP import Const
from fastNLP import DataSet
from fastNLP.core.predictor import Predictor
from myClue.tools.serialize import load_serialize_obj  # noqa
from myClue.tools.file import read_json_file_iter  # noqa

In [9]:
model_path = '/content/drive/My Drive/game/UCAS_NLP_TC/models/03_train_textcnn_words_split'
char_vocab_pkl_file = os.path.join(model_path, 'vocab_char.pkl')
target_vocab_pkl_file = os.path.join(model_path, 'target_char.pkl')
# 获取最新模型文件
model_list = [file_name for file_name in os.listdir(model_path) if 'best_' in file_name]
model_list.sort(key=lambda fn: os.path.getmtime(os.path.join(model_path, fn)) if not os.path.isdir(os.path.join(model_path, fn)) else 0)
model_name = os.path.join(model_path, model_list[-1])
print(model_name)

/content/drive/My Drive/game/UCAS_NLP_TC/models/03_train_textcnn_words_split/best_CNNText_f_2020-07-05-04-47-26-924614


In [10]:
logger.warn('开始加载模型')
model = torch.load(model_name)
model.eval()
logger.info('模型加载完毕:\n{}'.format(model))
logger.warn('获取词典')
char_vocab = load_serialize_obj(char_vocab_pkl_file)
logger.info('char_vocab:{}'.format(char_vocab))
target_vocab = load_serialize_obj(target_vocab_pkl_file)
logger.info('target_vocab:{}'.format(target_vocab))
logger.warn('加载测试数据')

2020-07-05 04:49:44 W [<ipython-input-10-982aee047e1d>:1] 开始加载模型
2020-07-05 04:49:44 I [<ipython-input-10-982aee047e1d>:4] 模型加载完毕:
CNNText(
  (embed): Embedding(
    (embed): StaticEmbedding(
      (dropout_layer): Dropout(p=0, inplace=False)
      (embedding): Embedding(96392, 50, padding_idx=0)
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (conv_pool): ConvMaxpool(
    (convs): ModuleList(
      (0): Conv1d(50, 30, kernel_size=(1,), stride=(1,), bias=False)
      (1): Conv1d(50, 40, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (2): Conv1d(50, 50, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
    )
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=120, out_features=5, bias=True)
)
2020-07-05 04:49:44 W [<ipython-input-10-982aee047e1d>:5] 获取词典
2020-07-05 04:49:45 I [<ipython-input-10-982aee047e1d>:7] char_vocab:Vocabulary(['近日', '，', '胡杏儿', '以', '品牌']...)
2020-07-05 04:49:45 I [<ipython-input-10-982aee047e1d>:9] target_voc

In [11]:
predict_output_json_file_name = '/content/drive/My Drive/game/UCAS_NLP_TC/pred_data/03_train_textcnn_words_split.json'

In [12]:
from myClue.tools.file import read_file_texts  # noqa
def news_content_process(news_content):
    """"数据转换处理"""
    words = news_content.split(' ')
    step = 50
    splited_words = [words[i:i+step] for i in range(0, len(words), step)]
    return splited_words

In [13]:
import json
from collections import defaultdict
from tqdm import tqdm
predictor = Predictor(model)
with codecs.open(predict_output_json_file_name, mode='w', encoding='utf8') as fw_json:
  texts = read_file_texts('/content/drive/My Drive/game/UCAS_NLP_TC/data/testdata.txt')
  for row_text in tqdm(texts):
    label, text = row_text.split('\t')
    news_content_words = news_content_process(text)
    predict_label_dict = defaultdict(int)
    for words in news_content_words:
      if len(words) < 10:
        continue
      test_data = [words]
      dataset = DataSet({Const.INPUT: test_data})
      dataset.add_seq_len(field_name=Const.INPUT)
      dataset.set_input(Const.INPUT, Const.INPUT_LEN)
      char_vocab.index_dataset(dataset, field_name=Const.INPUT)
      batch_output = predictor.predict(data=dataset, seq_len_field_name=Const.INPUT_LEN)
      # topk概率
      top_k_predict = batch_output.get('top_k_predict')[0]
      top_k_prob = batch_output.get('top_k_prob')[0]
      predict_results = list()
      for label_id, prob in zip(top_k_predict, top_k_prob):
        predict_label_dict[target_vocab.to_word(label_id)] += prob
    # 组装成所需格式
    row_data = {}
    row_data['label'] = label
    row_data['pred_label'] = predict_label_dict
    row_data['text'] = text
    fw_json.write('{}\n'.format(json.dumps(row_data, ensure_ascii=False)))

100%|██████████| 500/500 [00:13<00:00, 35.72it/s]


In [14]:
! head -2 /content/drive/My\ Drive/game/UCAS_NLP_TC/pred_data/03_train_textcnn_words_split.json

{"label": "EnterSports", "pred_label": {"EnterSports": 9.711933672428131, "Government": 0.04660993133438751, "Military": 0.17345445664250292, "Technology": 0.051570869003626285, "Economics": 0.016430955281975912}, "text": "U23 亚洲杯 小组赛 第二 轮 ， 中国 U23 国家队 在 完全 占据场 上 优势 的 情况 下 ， 无奈 久 攻 不 下 ， 0-1 不敌 乌兹别克队 ， 小组赛 前 两 轮 一 胜 一 负 出场 前景 成疑 。 但 虽败犹荣 的 是 ， U23 国 足在 早早 丢球 的 情况 下 ， 并没有 自乱阵脚 ， 在 逆风 球 中 敢 打 敢 拼 ， 若 不 是 临门一脚 火候 欠缺 ， 或许 至少 能 拿到 一 分 。本 届 U23 亚洲杯 ， 在 首 战 对阵 阿曼 的 比赛 中 ， 中国队 U23 的 小伙子们 不但 取得 了 亚洲杯 历史 首 胜 ， 更 是 表现 出 难能可贵 的 精气 神儿 ， 令 人 耳目一新 。 第二 轮 对阵 乌兹别克斯坦 ， 中国 U23 国家队 迎来 真正 考验 ， 阿曼队 虽然 是 预选赛 力压 伊朗队 的 “ 黑马 ” ， 但 整体 实力 在 亚洲 难 言 一流 ， 而 乌兹别克斯坦 各 个 级别 的 国 字号 球队 都 是 中国队 的 老 对手 ， 实力 也 更 胜 一 筹 。果然 在 比赛 开始 后 仅仅 第14 分钟 ， 乌兹别克斯坦队 就 抓住 一 次 中国队 禁区 解围不当 率先 取得 领先 ， 但是 瑕不掩瑜 的 是 ， 中国队 U23 的 小伙子们 ， 在 踢 逆风 球 的 情况 下 ， 并没有 自乱阵脚 ， 而 是 展现 出 强烈 的 求 胜 欲望 ， 尤其是 下 半场 ， 比赛 的 局面 完全 被 中国队 所 掌控 ， 就 连 央视 评论员 都 感叹 这 是 一 场 中国队 的 进攻 演练 ， 无论是 杨立瑜 的 包抄 打门 ， 还是 韦世豪 的 长途 奔袭 后 的 劲射 ， 都 险些 洞穿 对手 大门 。虽然 遗憾 的 是 ， 中国队 最终 还 是 未 能 在 比赛 

结果评测

In [15]:
with codecs.open(predict_output_json_file_name, mode='r', encoding='utf8') as fr:
  all_count = 0
  right_count = 0
  for line in fr:
    line = line.strip()
    if len(line) == 0:
      continue
    row_data = json.loads(line)
    all_count += 1
    pred_label_json = row_data['pred_label']
    label_list = sorted(pred_label_json.items(), key=lambda d: d[1], reverse=True)
    if row_data['label'] == label_list[0][0]:
      right_count += 1
  print('acc:{}/{}={}'.format(right_count, all_count, round(right_count/all_count, 4)))

acc:401/500=0.802
